# 📊 서울시 편의점 매출 완벽 통계분석
## "왜 어떤 편의점은 잘 팔리고 어떤 곳은 안 팔릴까?"

---

### 🎯 분석 목표
- **종속변수(Y)**: 편의점 매출
- **독립변수(X)**: 유동인구, 점포수, 상권유형(골목/발달/전통시장)

### 📋 분석 목차
1. 데이터 로드 및 병합
2. 상관분석 (유동인구 많으면 매출 높아?)
3. 다중회귀분석 (뭐가 제일 영향 커?)
4. VIF 다중공선성 (변수들 겹치는 거 없어?)
5. 잔차분석 (모델 믿을 만해?)
6. 더빈-왓슨 검정 (자기상관 없어?)
7. Cook's Distance (이상한 데이터 있어?)
8. ANOVA 분산분석 (상권유형별 매출 차이 있어?)
9. Tukey 사후검정 (어디랑 어디가 차이나?)
10. 변수선택법 (최적 변수 조합은?)
11. 신뢰구간 (회귀계수 얼마나 확실해?)
12. 최종 결론 및 발표 스토리

---
## 1. 라이브러리 및 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor, OLSInfluence
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("✅ 라이브러리 로드 완료!")

In [ ]:
# === 매출 데이터 로드 ===
df_sales_2022 = pd.read_csv('data/sales_2022.csv', encoding='cp949')
df_sales_2023 = pd.read_csv('data/sales_2023.csv', encoding='cp949')
df_sales_2024 = pd.read_csv('data/sales_2024.csv', encoding='cp949')
df_sales = pd.concat([df_sales_2022, df_sales_2023, df_sales_2024], ignore_index=True)
df_sales = df_sales[df_sales['서비스_업종_코드_명'] == '편의점'].copy()
print(f"✅ 매출 데이터: {len(df_sales):,}개")

# === 점포수 데이터 로드 ===
df_store_2022 = pd.read_csv('data/store_2022.csv', encoding='cp949')
df_store_2023 = pd.read_csv('data/store_2023.csv', encoding='cp949')
df_store_2024 = pd.read_csv('data/store_2024.csv', encoding='cp949')
df_store = pd.concat([df_store_2022, df_store_2023, df_store_2024], ignore_index=True)
df_store = df_store[df_store['서비스_업종_코드_명'] == '편의점'].copy()
df_store = df_store[['기준_년분기_코드', '행정동_코드', '점포_수']]
print(f"✅ 점포 데이터: {len(df_store):,}개")

# === 유동인구 데이터 로드 ===
df_pop = pd.read_csv('data/floating_population.csv', encoding='cp949')
df_pop = df_pop[df_pop['기준_년분기_코드'].between(20221, 20244)].copy()
df_pop = df_pop[['기준_년분기_코드', '행정동_코드', '총_유동인구_수']]
print(f"✅ 유동인구 데이터: {len(df_pop):,}개")

# === 상권유형 데이터 로드 ===
df_type = pd.read_csv('data/district_type.csv', encoding='cp949')
df_type_pivot = df_type.groupby(['행정동_코드', '상권_구분_코드_명']).size().unstack(fill_value=0).reset_index()
df_type_pivot.columns.name = None
print(f"✅ 상권유형 데이터: {len(df_type_pivot):,}개 행정동")

In [ ]:
# === 데이터 병합 ===
df = pd.merge(df_sales, df_store, on=['기준_년분기_코드', '행정동_코드'], how='inner')
print(f"매출 + 점포: {len(df):,}개")

df = pd.merge(df, df_pop, on=['기준_년분기_코드', '행정동_코드'], how='inner')
print(f"+ 유동인구: {len(df):,}개")

df = pd.merge(df, df_type_pivot, on='행정동_코드', how='left')
df = df.fillna(0)
print(f"+ 상권유형: {len(df):,}개")

# 분석용 데이터프레임 생성
df_analysis = df[['기준_년분기_코드', '행정동_코드', '행정동_코드_명',
                  '당월_매출_금액', '점포_수', '총_유동인구_수']].copy()

# 상권유형 컬럼 추가 (컬럼명이 다를 수 있어서 안전하게 처리)
for col in ['골목상권', '발달상권', '전통시장', '관광특구']:
    if col in df.columns:
        df_analysis[col] = df[col]
    else:
        df_analysis[col] = 0

# 컬럼명 간소화
df_analysis = df_analysis.rename(columns={
    '당월_매출_금액': '매출',
    '점포_수': '점포수',
    '총_유동인구_수': '유동인구'
})

print(f"\n✅ 최종 분석 데이터: {len(df_analysis):,}개")
print(f"컬럼: {df_analysis.columns.tolist()}")

In [ ]:
# === 데이터 정합성 검증 ===
print("📋 데이터 정합성 검증")
print("=" * 50)
print(f"\n[데이터 규모]")
print(f"  - 총 데이터 수: {len(df_analysis):,}개")
print(f"  - 행정동 수: {df_analysis['행정동_코드'].nunique()}개")
print(f"  - 분기 수: {df_analysis['기준_년분기_코드'].nunique()}개")

print(f"\n[결측치 확인]")
null_counts = df_analysis.isnull().sum()
if null_counts.sum() == 0:
    print("  ✅ 결측치 없음!")
else:
    print(null_counts[null_counts > 0])

print(f"\n[주요 변수 기술통계]")
print(f"  - 매출 평균: {df_analysis['매출'].mean()/1e8:.2f}억원")
print(f"  - 매출 중앙값: {df_analysis['매출'].median()/1e8:.2f}억원")
print(f"  - 유동인구 평균: {df_analysis['유동인구'].mean()/1e6:.2f}백만명")
print(f"  - 점포수 평균: {df_analysis['점포수'].mean():.1f}개")

---
## 2. 상관분석 📈
### 💡 질문: "유동인구 많으면 매출 높아?"

**분석 목적**: 변수들 간의 관계 강도와 방향 확인

**해석 기준**:
- |r| > 0.7: 강한 상관
- |r| 0.4~0.7: 중간 상관
- |r| < 0.4: 약한 상관

**발표 스토리**: "유동인구와 매출의 상관계수는 r=0.XX로, [강한/중간/약한] 양의 상관관계를 보입니다. 즉, 유동인구가 많을수록 매출이 높아지는 경향이 있습니다."

In [ ]:
# 상관계수 계산
corr_cols = ['매출', '유동인구', '점포수', '골목상권', '발달상권', '전통시장']
corr_matrix = df_analysis[corr_cols].corr()

print("📊 상관계수 행렬")
print("=" * 70)
print(corr_matrix.round(3))

In [ ]:
# 히트맵 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdYlBu_r', center=0, fmt='.3f',
            square=True, linewidths=0.5, cbar_kws={'shrink': 0.8})
plt.title('변수 간 상관관계 히트맵', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 매출과 각 변수의 상관관계 + 유의성 검정
print("📊 매출과 각 변수의 상관관계 (유의성 검정 포함)")
print("=" * 60)
print(f"{'변수':<12} {'상관계수':>10} {'p-value':>15} {'유의성':>8} {'해석':>15}")
print("-" * 60)

for col in ['유동인구', '점포수', '골목상권', '발달상권', '전통시장']:
    corr, p_val = stats.pearsonr(df_analysis['매출'], df_analysis[col])
    sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else "ns"
    
    if abs(corr) > 0.7:
        strength = "강한 상관"
    elif abs(corr) > 0.4:
        strength = "중간 상관"
    else:
        strength = "약한 상관"
    
    print(f"{col:<12} {corr:>10.4f} {p_val:>15.2e} {sig:>8} {strength:>15}")

print("\n*** p<0.001, ** p<0.01, * p<0.05, ns: 유의하지 않음")

---
## 3. 다중회귀분석 📉
### 💡 질문: "유동인구, 점포수, 상권유형 중 뭐가 제일 영향 커?"

**분석 목적**: 여러 독립변수가 매출에 미치는 영향 동시 분석

**해석 기준**:
- R²: 모델 설명력 (0~1, 높을수록 좋음)
- p-value < 0.05: 해당 변수가 유의미함
- 계수(coef): 1단위 증가 시 매출 변화량

**발표 스토리**: "다중회귀분석 결과, 모델의 설명력(R²)은 XX%입니다. 유의한 변수는 XX, XX이며, 특히 XX가 1단위 증가할 때 매출이 약 XX억원 증가합니다."

In [ ]:
# 다중회귀분석 실행
X_cols = ['유동인구', '점포수', '골목상권', '발달상권', '전통시장']
X = df_analysis[X_cols]
X = sm.add_constant(X)
y = df_analysis['매출']

model = sm.OLS(y, X).fit()

print("📊 다중회귀분석 결과")
print("=" * 70)
print(model.summary())

In [ ]:
# 회귀계수 해석 (단위 변환)
print("\n📈 회귀계수 해석 (쉬운 해석)")
print("=" * 70)
print(f"\n모델 설명력 (R²): {model.rsquared:.4f} → 매출 변동의 {model.rsquared*100:.1f}%를 설명")
print(f"모델 유의성 (F-test p-value): {model.f_pvalue:.2e} → {'유의함' if model.f_pvalue < 0.05 else '유의하지 않음'}")

print("\n[변수별 영향력]")
print("-" * 70)
for var in X_cols:
    coef = model.params[var]
    pval = model.pvalues[var]
    sig = "✅ 유의함" if pval < 0.05 else "❌ 유의하지 않음"
    
    # 단위 변환해서 해석
    if var == '유동인구':
        # 유동인구 100만명 증가 시
        effect = coef * 1e6 / 1e8
        print(f"{var}: 100만명 증가 시 매출 {effect:+.2f}억원 (p={pval:.4f}) {sig}")
    elif var == '점포수':
        # 점포 1개 증가 시
        effect = coef / 1e8
        print(f"{var}: 1개 증가 시 매출 {effect:+.2f}억원 (p={pval:.4f}) {sig}")
    else:
        # 상권 1개 증가 시
        effect = coef / 1e8
        print(f"{var}: 1개 증가 시 매출 {effect:+.2f}억원 (p={pval:.4f}) {sig}")

In [ ]:
# 회귀계수 시각화 (억원 단위로 변환)
coef_df = pd.DataFrame({
    '변수': X_cols,
    '계수(억원)': [model.params[col]/1e8 if col != '유동인구' else model.params[col]*1e6/1e8 for col in X_cols],
    'p-value': [model.pvalues[col] for col in X_cols]
})

plt.figure(figsize=(10, 5))
colors = ['green' if p < 0.05 else 'gray' for p in coef_df['p-value']]
bars = plt.barh(coef_df['변수'], coef_df['계수(억원)'], color=colors)
plt.axvline(x=0, color='black', linestyle='-', linewidth=1)
plt.xlabel('회귀계수 (억원 단위)')
plt.title('변수별 회귀계수\n(녹색=유의함 p<0.05, 회색=유의하지않음)', fontsize=12)
plt.tight_layout()
plt.show()

print("\n💡 해석: 계수가 양수(+)면 매출 증가, 음수(-)면 매출 감소 효과")
print("   골목상권이 음수인 이유: 골목상권은 주거지역 근처 작은 상권이라 상대적으로 매출이 낮음")

---
## 4. VIF 다중공선성 검정 🔍
### 💡 질문: "독립변수들끼리 겹치는 거 없어?"

**분석 목적**: 독립변수 간 상관이 너무 높으면 회귀계수 신뢰 불가

**해석 기준**:
- VIF < 5: 문제 없음 ✅
- VIF 5~10: 주의 필요 ⚠️
- VIF > 10: 심각한 다중공선성 ❌

**발표 스토리**: "다중공선성 검정(VIF) 결과, 모든 변수의 VIF가 5 미만으로 독립변수 간 심각한 상관은 없었습니다."

In [ ]:
# VIF 계산
X_vif = df_analysis[X_cols]

vif_data = pd.DataFrame()
vif_data['변수'] = X_cols
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(len(X_cols))]
vif_data['판정'] = vif_data['VIF'].apply(lambda x: '❌ 심각' if x > 10 else '⚠️ 주의' if x > 5 else '✅ 양호')

print("📊 다중공선성 검정 (VIF)")
print("=" * 50)
print("VIF 해석 기준:")
print("  - VIF < 5: 문제 없음")
print("  - VIF 5~10: 주의 필요")
print("  - VIF > 10: 심각한 다중공선성")
print("-" * 50)
print(vif_data.to_string(index=False))

In [ ]:
# VIF 시각화
plt.figure(figsize=(10, 5))
colors = ['red' if v > 10 else 'orange' if v > 5 else 'green' for v in vif_data['VIF']]
bars = plt.barh(vif_data['변수'], vif_data['VIF'], color=colors)
plt.axvline(x=5, color='orange', linestyle='--', linewidth=2, label='VIF=5 (주의 기준)')
plt.axvline(x=10, color='red', linestyle='--', linewidth=2, label='VIF=10 (심각 기준)')
plt.xlabel('VIF')
plt.title('다중공선성 검정 (VIF)', fontsize=14, fontweight='bold')
plt.legend()
plt.tight_layout()
plt.show()

---
## 5. 잔차분석 📐
### 💡 질문: "우리 모델 믿을 만해?"

**분석 목적**: 회귀모델의 가정(정규성, 등분산성) 충족 여부 확인

**확인 항목**:
- 정규성: 잔차가 정규분포를 따르는가? (Shapiro-Wilk)
- 등분산성: 잔차 분산이 일정한가? (Breusch-Pagan)

**발표 스토리**: "잔차분석 결과, 정규성은 [만족/불만족]하고 등분산성은 [만족/불만족]합니다. [모델이 적합함/추가 개선 필요]"

In [ ]:
# 잔차 계산
residuals = model.resid
fitted = model.fittedvalues

# 4개 잔차 플롯
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. 잔차 vs 적합값 (등분산성)
axes[0, 0].scatter(fitted/1e8, residuals/1e8, alpha=0.3, s=10)
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('적합값 (억원)')
axes[0, 0].set_ylabel('잔차 (억원)')
axes[0, 0].set_title('① 잔차 vs 적합값 (등분산성 확인)\n→ 점들이 0 주변에 고르게 퍼져야 함')

# 2. 잔차 히스토그램 (정규성)
axes[0, 1].hist(residuals/1e8, bins=50, edgecolor='black', alpha=0.7, density=True)
x_range = np.linspace(residuals.min()/1e8, residuals.max()/1e8, 100)
axes[0, 1].plot(x_range, stats.norm.pdf(x_range, residuals.mean()/1e8, residuals.std()/1e8), 'r-', linewidth=2, label='정규분포')
axes[0, 1].set_xlabel('잔차 (억원)')
axes[0, 1].set_ylabel('밀도')
axes[0, 1].set_title('② 잔차 분포 (정규성 확인)\n→ 종 모양이어야 함')
axes[0, 1].legend()

# 3. Q-Q Plot (정규성)
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('③ Q-Q Plot (정규성 확인)\n→ 점들이 빨간 선 위에 있어야 함')

# 4. 잔차 vs 순서 (독립성)
axes[1, 1].plot(residuals/1e8, alpha=0.5, linewidth=0.5)
axes[1, 1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('관측치 순서')
axes[1, 1].set_ylabel('잔차 (억원)')
axes[1, 1].set_title('④ 잔차 vs 순서 (독립성 확인)\n→ 패턴이 없어야 함')

plt.tight_layout()
plt.show()

In [ ]:
# 정규성 검정 (Shapiro-Wilk)
sample_size = min(5000, len(residuals))
sample_residuals = residuals.sample(sample_size, random_state=42)
shapiro_stat, shapiro_p = stats.shapiro(sample_residuals)

# 등분산성 검정 (Breusch-Pagan)
bp_stat, bp_p, _, _ = het_breuschpagan(residuals, X)

print("📊 잔차 가정 검정")
print("=" * 60)
print(f"\n[정규성 검정 - Shapiro-Wilk]")
print(f"  H0: 잔차가 정규분포를 따른다")
print(f"  통계량: {shapiro_stat:.4f}")
print(f"  p-value: {shapiro_p:.4e}")
print(f"  결과: {'✅ 정규성 만족 (p≥0.05)' if shapiro_p >= 0.05 else '⚠️ 정규성 불만족 (p<0.05)'}")

print(f"\n[등분산성 검정 - Breusch-Pagan]")
print(f"  H0: 잔차의 분산이 일정하다")
print(f"  통계량: {bp_stat:.4f}")
print(f"  p-value: {bp_p:.4e}")
print(f"  결과: {'✅ 등분산성 만족 (p≥0.05)' if bp_p >= 0.05 else '⚠️ 이분산성 존재 (p<0.05)'}")

---
## 6. 더빈-왓슨 검정 🔄
### 💡 질문: "분기별 데이터가 서로 영향 안 줘?"

**분석 목적**: 잔차 간 자기상관 여부 확인 (시계열 데이터에서 중요)

**해석 기준**:
- DW ≈ 2: 자기상관 없음 ✅
- DW < 1.5: 양의 자기상관 ⚠️
- DW > 2.5: 음의 자기상관 ⚠️

**발표 스토리**: "더빈-왓슨 통계량은 XX로, 2에 가까워 잔차 간 자기상관이 없습니다."

In [ ]:
# 더빈-왓슨 검정
dw_stat = durbin_watson(residuals)

print("📊 더빈-왓슨 검정 (자기상관)")
print("=" * 50)
print(f"DW 통계량: {dw_stat:.4f}")
print()
print("해석 기준:")
print("  - DW ≈ 2: 자기상관 없음")
print("  - DW < 1.5: 양의 자기상관")
print("  - DW > 2.5: 음의 자기상관")
print()

if 1.5 <= dw_stat <= 2.5:
    print(f"결과: ✅ 자기상관 없음 (DW={dw_stat:.2f} ≈ 2)")
elif dw_stat < 1.5:
    print(f"결과: ⚠️ 양의 자기상관 존재 (DW={dw_stat:.2f} < 1.5)")
else:
    print(f"결과: ⚠️ 음의 자기상관 존재 (DW={dw_stat:.2f} > 2.5)")

In [ ]:
# 더빈-왓슨 시각화
plt.figure(figsize=(10, 3))
plt.axvspan(0, 1.5, alpha=0.3, color='red', label='양의 자기상관')
plt.axvspan(1.5, 2.5, alpha=0.3, color='green', label='자기상관 없음')
plt.axvspan(2.5, 4, alpha=0.3, color='orange', label='음의 자기상관')
plt.axvline(x=dw_stat, color='blue', linewidth=4, label=f'DW = {dw_stat:.2f}')
plt.axvline(x=2, color='black', linestyle='--', linewidth=1, label='이상적 값 (DW=2)')
plt.xlim(0, 4)
plt.xlabel('Durbin-Watson 통계량')
plt.title('더빈-왓슨 검정 결과', fontsize=12)
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

---
## 7. Cook's Distance 🎯
### 💡 질문: "결과 왜곡하는 이상한 데이터 있어?"

**분석 목적**: 회귀 결과에 과도한 영향을 미치는 이상치 탐지

**해석 기준**:
- Cook's D > 4/n: 영향력 있는 관측치
- Cook's D > 1: 매우 영향력 큰 관측치 (제거 고려)

**발표 스토리**: "Cook's Distance 분석 결과, 영향력 있는 이상치는 전체의 XX%로, 주로 강남 등 고매출 지역에서 나타났습니다."

In [ ]:
# Cook's Distance 계산
influence = OLSInfluence(model)
cooks_d = influence.cooks_distance[0]

# 임계값
n = len(df_analysis)
threshold = 4 / n

# 영향력 있는 관측치
influential_idx = np.where(cooks_d > threshold)[0]

print("📊 Cook's Distance 분석")
print("=" * 50)
print(f"임계값 (4/n): {threshold:.6f}")
print(f"영향력 있는 관측치 수: {len(influential_idx)}개 ({len(influential_idx)/n*100:.1f}%)")
print(f"Cook's D 최대값: {cooks_d.max():.4f}")
print(f"Cook's D > 1인 관측치: {sum(cooks_d > 1)}개")

In [ ]:
# Cook's Distance 시각화
plt.figure(figsize=(12, 5))
plt.stem(range(len(cooks_d)), cooks_d, markerfmt=',', basefmt=' ')
plt.axhline(y=threshold, color='red', linestyle='--', linewidth=2, label=f'임계값 = {threshold:.4f}')
plt.xlabel('관측치 인덱스')
plt.ylabel("Cook's Distance")
plt.title("Cook's Distance (영향력 있는 이상치 탐지)", fontsize=12)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 가장 영향력 큰 관측치 TOP 10
top_influential = np.argsort(cooks_d)[-10:][::-1]

print("\n🔝 가장 영향력 큰 관측치 TOP 10")
print("=" * 70)
print(f"{'순위':<4} {'행정동':<15} {'매출(억원)':<12} {'유동인구(만명)':<12} {'Cook D':<10}")
print("-" * 70)
for rank, idx in enumerate(top_influential, 1):
    row = df_analysis.iloc[idx]
    print(f"{rank:<4} {row['행정동_코드_명']:<15} {row['매출']/1e8:<12.1f} {row['유동인구']/1e4:<12.1f} {cooks_d[idx]:<10.4f}")

---
## 8. ANOVA 분산분석 📊
### 💡 질문: "골목상권 vs 발달상권 vs 전통시장 매출 차이 있어?"

**분석 목적**: 3개 이상 그룹 간 평균 차이 검정

**해석 기준**:
- p-value < 0.05: 그룹 간 유의한 차이 있음
- F값이 클수록 그룹 간 차이가 큼

**발표 스토리**: "ANOVA 분석 결과, 상권유형에 따라 매출에 통계적으로 유의한 차이가 있었습니다 (F=XX, p<0.05)."

In [ ]:
# 주요 상권유형 결정
def get_main_type(row):
    types = {'골목상권': row['골목상권'], '발달상권': row['발달상권'], '전통시장': row['전통시장']}
    if max(types.values()) == 0:
        return '기타'
    return max(types, key=types.get)

df_analysis['주요_상권유형'] = df_analysis.apply(get_main_type, axis=1)

# 상권유형별 기술통계
print("📊 상권유형별 매출 기술통계")
print("=" * 60)
type_stats = df_analysis.groupby('주요_상권유형')['매출'].agg(['count', 'mean', 'std', 'min', 'max'])
type_stats['mean'] = type_stats['mean'] / 1e8
type_stats['std'] = type_stats['std'] / 1e8
type_stats['min'] = type_stats['min'] / 1e8
type_stats['max'] = type_stats['max'] / 1e8
type_stats.columns = ['개수', '평균(억원)', '표준편차', '최소', '최대']
print(type_stats.round(2))

In [ ]:
# 상권유형별 박스플롯
plt.figure(figsize=(10, 6))
order = ['발달상권', '골목상권', '전통시장', '기타']
available_types = [t for t in order if t in df_analysis['주요_상권유형'].unique()]
sns.boxplot(data=df_analysis, x='주요_상권유형', y='매출', order=available_types)
plt.title('상권유형별 매출 분포', fontsize=14)
plt.xlabel('상권유형')
plt.ylabel('매출 (원)')
plt.tight_layout()
plt.show()

In [ ]:
# One-way ANOVA
groups = [group['매출'].values for name, group in df_analysis.groupby('주요_상권유형')]
f_stat, anova_p = stats.f_oneway(*groups)

print("📊 ANOVA 분산분석 결과")
print("=" * 50)
print(f"H0: 모든 상권유형의 평균 매출이 같다")
print(f"H1: 적어도 하나의 상권유형 평균 매출이 다르다")
print()
print(f"F-통계량: {f_stat:.4f}")
print(f"p-value: {anova_p:.4e}")
print()
if anova_p < 0.05:
    print("결과: ✅ 상권유형별 매출에 유의한 차이가 있음 (p < 0.05)")
    print("      → 어떤 상권유형 간에 차이가 있는지 Tukey 사후검정 필요")
else:
    print("결과: ❌ 상권유형별 매출에 유의한 차이 없음 (p ≥ 0.05)")

---
## 9. Tukey 사후검정 🔬
### 💡 질문: "정확히 어디랑 어디가 차이나?"

**분석 목적**: ANOVA에서 차이가 있다면, 어떤 그룹 간에 차이가 있는지 확인

**해석 기준**:
- p-adj < 0.05: 해당 두 그룹 간 유의한 차이 있음

**발표 스토리**: "Tukey 사후검정 결과, 발달상권이 골목상권보다 평균 XX억원 높았으며 이 차이는 통계적으로 유의했습니다."

In [ ]:
# Tukey HSD 사후검정
tukey_result = pairwise_tukeyhsd(df_analysis['매출'], df_analysis['주요_상권유형'], alpha=0.05)

print("📊 Tukey HSD 사후검정 결과")
print("=" * 70)
print(tukey_result)

In [ ]:
# Tukey 결과 시각화
fig, ax = plt.subplots(figsize=(10, 6))
tukey_result.plot_simultaneous(ax=ax)
plt.title('Tukey HSD 사후검정 (95% 신뢰구간)\n→ 신뢰구간이 겹치지 않으면 유의한 차이', fontsize=12)
plt.xlabel('매출 (원)')
plt.tight_layout()
plt.show()

In [ ]:
# 사후검정 결과 해석
print("\n📋 사후검정 결과 해석 (쉬운 해석)")
print("=" * 60)

tukey_df = pd.DataFrame(data=tukey_result._results_table.data[1:], 
                        columns=tukey_result._results_table.data[0])

for idx, row in tukey_df.iterrows():
    g1, g2 = row['group1'], row['group2']
    diff = float(row['meandiff']) / 1e8  # 억원 단위
    reject = row['reject']
    
    if reject:
        higher = g1 if diff < 0 else g2
        lower = g2 if diff < 0 else g1
        print(f"✅ {g1} vs {g2}: 유의한 차이 있음")
        print(f"   → {higher}가 {lower}보다 평균 {abs(diff):.1f}억원 높음")
    else:
        print(f"❌ {g1} vs {g2}: 유의한 차이 없음")
    print()

---
## 10. 변수선택법 (Stepwise) 🎛️
### 💡 질문: "최적의 변수 조합은 뭐야?"

**분석 목적**: 가장 설명력 높은 변수 조합 찾기

**방법**: 모든 변수 조합의 Adjusted R² 비교

**발표 스토리**: "변수 선택 분석 결과, [유동인구, 점포수] 조합이 가장 높은 설명력(Adj.R²=XX)을 보였습니다."

In [ ]:
# 모든 변수 조합 비교
from itertools import combinations

all_vars = ['유동인구', '점포수', '골목상권', '발달상권', '전통시장']
results = []

for k in range(1, len(all_vars) + 1):
    for combo in combinations(all_vars, k):
        X_temp = sm.add_constant(df_analysis[list(combo)])
        model_temp = sm.OLS(y, X_temp).fit()
        results.append({
            '변수': ', '.join(combo),
            '변수개수': k,
            'R²': model_temp.rsquared,
            'Adj_R²': model_temp.rsquared_adj,
            'AIC': model_temp.aic
        })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('Adj_R²', ascending=False)

print("📊 변수 조합별 모델 성능 (상위 10개)")
print("=" * 80)
print(results_df.head(10).to_string(index=False))

In [ ]:
# 최적 모델 확인
best_vars = results_df.iloc[0]['변수']
best_adj_r2 = results_df.iloc[0]['Adj_R²']

print("\n🏆 최적 모델")
print("=" * 50)
print(f"최적 변수 조합: {best_vars}")
print(f"Adjusted R²: {best_adj_r2:.4f} ({best_adj_r2*100:.1f}%)")

# 변수 개수별 최고 성능
best_by_k = results_df.groupby('변수개수')['Adj_R²'].max()

plt.figure(figsize=(8, 5))
plt.plot(best_by_k.index, best_by_k.values, marker='o', linewidth=2, markersize=10)
plt.xlabel('변수 개수')
plt.ylabel('Adjusted R²')
plt.title('변수 개수에 따른 최고 Adjusted R²', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 11. 회귀계수 신뢰구간 📏
### 💡 질문: "회귀계수가 얼마나 확실해?"

**분석 목적**: 회귀계수의 불확실성 범위 확인

**해석**:
- 95% 신뢰구간에 0이 포함되면 유의하지 않음
- 신뢰구간이 좁을수록 추정이 정확함

**발표 스토리**: "유동인구의 회귀계수는 XX이며, 95% 신뢰구간은 [XX, XX]입니다. 0을 포함하지 않아 유의합니다."

In [ ]:
# 신뢰구간 추출
conf_int = model.conf_int(alpha=0.05)
conf_int.columns = ['하한 (2.5%)', '상한 (97.5%)']
conf_int['계수'] = model.params
conf_int['p-value'] = model.pvalues
conf_int = conf_int[['계수', '하한 (2.5%)', '상한 (97.5%)', 'p-value']]

print("📊 회귀계수 95% 신뢰구간")
print("=" * 70)
print(conf_int.round(4))

In [ ]:
# 신뢰구간 시각화 (억원 단위로 변환)
fig, ax = plt.subplots(figsize=(10, 6))

vars_to_plot = X_cols
y_pos = range(len(vars_to_plot))

for i, var in enumerate(vars_to_plot):
    # 단위 변환
    if var == '유동인구':
        scale = 1e6 / 1e8  # 100만명당 억원
    else:
        scale = 1 / 1e8  # 1개당 억원
    
    coef = model.params[var] * scale
    ci_low = conf_int.loc[var, '하한 (2.5%)'] * scale
    ci_high = conf_int.loc[var, '상한 (97.5%)'] * scale
    
    color = 'green' if model.pvalues[var] < 0.05 else 'gray'
    ax.plot([ci_low, ci_high], [i, i], color=color, linewidth=3)
    ax.scatter(coef, i, color=color, s=100, zorder=5)

ax.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax.set_yticks(y_pos)
ax.set_yticklabels(vars_to_plot)
ax.set_xlabel('회귀계수 (억원 단위)')
ax.set_title('회귀계수 95% 신뢰구간\n(녹색=유의 p<0.05, 회색=비유의, 빨간선=0)\n→ 0과 겹치지 않으면 유의함', fontsize=12)
plt.tight_layout()
plt.show()

---
# 📝 12. 최종 결론 및 발표 스토리

In [ ]:
# 결과 요약 변수
sig_vars = [var for var in X_cols if model.pvalues[var] < 0.05]
nonsig_vars = [var for var in X_cols if model.pvalues[var] >= 0.05]

print("="*80)
print("📝 완벽 통계분석 최종 결론")
print("="*80)

print(f"""
┌────────────────────────────────────────────────────────────────────┐
│  🎯 연구 질문: "왜 어떤 편의점은 잘 팔리고 어떤 곳은 안 팔릴까?"  │
└────────────────────────────────────────────────────────────────────┘

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 1. 상관분석 결과
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
→ 매출과 가장 높은 상관: 점포수 (r = {corr_matrix.loc['매출', '점포수']:.3f})
→ 해석: 점포수가 많을수록 (= 상권이 발달할수록) 매출이 높음

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 2. 다중회귀분석 결과  
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
→ 모델 설명력 (R²): {model.rsquared:.4f} ({model.rsquared*100:.1f}%)
→ 유의한 변수: {', '.join(sig_vars) if sig_vars else '없음'}
→ 유의하지 않은 변수: {', '.join(nonsig_vars) if nonsig_vars else '없음'}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 3. 다중공선성 (VIF)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
→ 최대 VIF: {vif_data['VIF'].max():.2f}
→ 판정: {'⚠️ 다중공선성 주의' if vif_data['VIF'].max() > 5 else '✅ 다중공선성 문제 없음'}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 4. 잔차분석
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
→ 정규성: {'✅ 만족' if shapiro_p >= 0.05 else '⚠️ 불만족'} (Shapiro p = {shapiro_p:.4e})
→ 등분산성: {'✅ 만족' if bp_p >= 0.05 else '⚠️ 이분산성'} (BP p = {bp_p:.4e})

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 5. 더빈-왓슨 (자기상관)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
→ DW 통계량: {dw_stat:.4f}
→ 판정: {'✅ 자기상관 없음' if 1.5 <= dw_stat <= 2.5 else '⚠️ 자기상관 존재'}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 6. Cook's Distance
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
→ 영향력 있는 이상치: {len(influential_idx)}개 ({len(influential_idx)/n*100:.1f}%)
→ 최대 Cook's D: {cooks_d.max():.4f}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 7. ANOVA (상권유형별 차이)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
→ F-통계량: {f_stat:.4f}, p-value: {anova_p:.4e}
→ 판정: {'✅ 상권유형별 매출 차이 유의함' if anova_p < 0.05 else '❌ 유의한 차이 없음'}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 8. 최적 변수 조합
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
→ 최적 변수: {best_vars}
→ Adjusted R²: {best_adj_r2:.4f} ({best_adj_r2*100:.1f}%)
""")

In [ ]:
print("""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🎤 발표 스토리 (대본 예시)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

"안녕하세요, 저희 팀은 '왜 어떤 편의점은 잘 팔리고 어떤 곳은 안 팔릴까?'라는 
질문에 답하기 위해 서울시 편의점 매출 데이터를 분석했습니다.

📊 [상관분석]
먼저 상관분석 결과, 점포수와 매출이 가장 높은 상관관계를 보였습니다.
이는 상권이 발달한 지역일수록 편의점 매출이 높다는 것을 의미합니다.

📊 [다중회귀분석]
다중회귀분석을 통해 유동인구, 점포수, 상권유형이 매출에 미치는 영향을 
동시에 분석했습니다. 모델의 설명력(R²)은 약 {0}%였습니다.

📊 [모델 검증]
다중공선성 검정(VIF) 결과 독립변수 간 심각한 상관은 없었고,
더빈-왓슨 검정 결과 자기상관도 없어 모델이 신뢰할 만합니다.

📊 [ANOVA 분석]
특히 ANOVA 분석 결과, 상권유형(골목/발달/전통시장)에 따라 
매출에 통계적으로 유의한 차이가 있었습니다.
Tukey 사후검정 결과, 발달상권의 매출이 가장 높았습니다.

📌 [결론]
결론적으로, 편의점 매출을 높이려면:
  1. 유동인구가 많은 지역 선택
  2. 발달상권 입지 고려
  3. 적정 점포 밀도 유지
가 중요한 요인임을 확인했습니다.

감사합니다."
""".format(model.rsquared*100))